In [2]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import rotation_matrix as rm
import time
import numpy as np
import rotation_matrix as rm

ModuleNotFoundError: No module named 'rtde_control'

In [13]:
def connect_robot(ip = "192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip) #IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

rtde_c, rtde_r, rtde_io_set = connect_robot()

In [3]:
import numpy as np


In [6]:
#samples = np.loadtxt(("../pouring_simulation/output/flask_test_samples.txt"))  # load the samples from the simulation

samples = np.loadtxt("../pouring_simulation/output/flask_test_samples.txt", dtype=str)

# load text file with the samples


In [8]:
print(samples[1])

['Sample' 'folder:' '../../output/CellFlask/Flask_65_1399_24,' 'Starting'
 'volume:' '62.72,' 'Poured' 'volume:' '56.285998799999994,' 'Stop'
 'angle:' '24.0,' 'Stop' 'time:' '1.4']


In [15]:


data = np.loadtxt('../pouring_simulation/output/CellFlask/Flask_50_1799_28/TCP.txt', delimiter=',', skiprows=1)

In [16]:
data.shape[0]

2123

In [17]:
#start_pos = [-0.32840330749723673,
# 0.6906975268273259,
# 0.25610656751907307,
# -1.2222948307378443,
# -1.1666822479428505,
# -1.218172452957232]

#rtde_c.moveL(start_pos, 0.5, 0.5)

In [18]:

# convert from inches to meters
data[:, 0] = data[:, 0] * 0.0254
data[:, 1] = data[:, 1] * 0.0254

In [19]:
data

array([[ 2.42233196e-08,  8.29891660e-05,  5.00000000e-04],
       [ 8.47816440e-08,  1.66002462e-04,  1.00000000e-03],
       [ 1.93786506e-07,  2.48991628e-04,  1.50000000e-03],
       ...,
       [ 2.42233196e-08, -8.29891660e-05,  0.00000000e+00],
       [ 2.42233196e-08, -8.29891660e-05,  0.00000000e+00],
       [ 2.42233196e-08, -8.29891660e-05,  0.00000000e+00]])

In [20]:
start_pos = rtde_r.getActualTCPPose()

In [21]:
start_pos

[-0.4058390226168195,
 0.13062551121065363,
 0.21988165930384806,
 -2.184084280634187,
 0.03481368238601551,
 2.2543938904225596]

In [22]:
start_pos = [-0.4057309121445867,
 0.12930964966911077,
 0.22103104593020784,
 2.1799156334772527,
 -0.02785233113705692,
 -2.261495615466219]

# start_pos for flask holder 1

In [23]:
filling_pos = [-0.4057147781327205,
 0.1261790155942795,
 0.22288981299466953,
 -1.6454642598425688,
 -0.7887050197628324,
 1.696551210550644]

In [24]:
# create list of positions
positions = []
for i in range(data.shape[0]):
    positions.append([-data[i,1], -data[i,0], 0.0, 0.0, 0.0, data[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups

In [32]:
positions_converted = []
for i in range(data.shape[0]):
    # if none of the data entries is 0
    if not (positions[i][0] == 0 or positions[i][1] == 0 or positions[i][5] == 0):
        positions_converted.append(rm.PoseTrans(start_pos, positions[i])) # transform from base to tool coordinates(positions[i]))

In [34]:
# get first 5 x and z positions 
positions = positions_converted[15:30]

# only get x, y, z positions
positions = [x[0:3] for x in positions]


#positions = [x[0:3:2] for x in positions]

# calculate distances between positions in meters
distances = []
for i in range(len(positions)-1):
    distances.append(np.linalg.norm(np.subtract(positions[i+1], positions[i])))

print(distances)

speed = []

# calculate speed for a frequency of 60 Hz in m/s
for i in range(len(distances)):
    speed.append(distances[i]*89)

print(speed)

# calculate average speed
avg_speed = np.mean(speed)
print("Speed in m/s: ", avg_speed)

[8.29921913733594e-05, 8.299260179693285e-05, 8.301578659637857e-05, 8.299338726925845e-05, 8.299387527323725e-05, 8.29941296158007e-05, 8.301751832107392e-05, 8.299521847825028e-05, 8.299565608266637e-05, 8.299610895782249e-05, 8.299673871629918e-05, 8.299722866405155e-05, 8.302076524648826e-05, 8.299843534002479e-05]
[0.007386305032228987, 0.007386341559927023, 0.007388405007077693, 0.007386411466964002, 0.007386454899318115, 0.007386477535806263, 0.007388559130575579, 0.007386574444564274, 0.007386613391357307, 0.007386653697246202, 0.007386709745750627, 0.007386753351100588, 0.007388848106937455, 0.007386860745262206]
Speed in m/s:  0.0073869977224368805


In [35]:
# get position of the first duplicate converted_position that is not position 0
for i in range(len(positions_converted)):
    if positions_converted[i] == positions_converted[0]:
        continue
    elif positions_converted[i] == positions_converted[i-1] == positions_converted[i-2] == positions_converted[i-3] == positions_converted[i-4]:
        print(i)
        break

# count the values that are the same as i
count = 0
for j in range(i, len(positions_converted)):
    if positions_converted[j] == positions_converted[i]:
        count += 1
    else:
        break
print(count)

980
105


In [36]:
# split positions_converted into two lists
positions_converted1 = positions_converted[0:i]
positions_converted2 = positions_converted[i:]

In [37]:
# print length of both lists
print(len(positions_converted1))
print(len(positions_converted2))

980
1081


In [38]:
positions

[[-0.40568197878797624, 0.12933167940080242, 0.22235796770119048],
 [-0.40567891667233624, 0.12933340199099866, 0.22244088549156008],
 [-0.4056758539734901, 0.12933517306216696, 0.22252380264983096],
 [-0.405672789996846, 0.12933698071193062, 0.22260674217764292],
 [-0.4056697262785348, 0.12933883652956502, 0.2226896582309827],
 [-0.405666661977323, 0.12934074080277547, 0.22277257365255487],
 [-0.4056635973849664, 0.12934266927837793, 0.22285548875857436],
 [-0.4056605313687811, 0.12934464647186575, 0.2229384260758619],
 [-0.4056574656109288, 0.12934667183322432, 0.22302133991867726],
 [-0.405654399415901, 0.12934873353626494, 0.22310425328766692],
 [-0.40565133278430876, 0.1293508315301956, 0.2231871661834931],
 [-0.4056482655695104, 0.12935297800509832, 0.22327007844722058],
 [-0.40564519791814757, 0.12935516077089107, 0.22335299023778454],
 [-0.4056421288426506, 0.12935739227996515, 0.2234359242392855],
 [-0.40563906017121176, 0.12935965984301598, 0.22351883492425614]]

In [29]:
rtde_c.moveL(filling_pos, 0.5, 0.5)

True

In [30]:
rtde_c.moveL(start_pos, 0.02, 0.1)

True

In [31]:
# 11,../../output/CellFlask/Flask_35_200_22,0.030000,22.000000,0.200000,34.032500,16.002501,15.885000,0.117500

In [39]:
velocity = avg_speed #0.5
acceleration = 1.5
blend_1 = 0.0
blend_i = 0.001
blend_3 = 0.0
#path_pose1 = [start_pos_flask2[0], start_pos_flask2[1], start_pos_flask2[2], start_pos_flask2[3], start_pos_flask2[4], start_pos_flask2[5], velocity, acceleration, blend_1]
path = []
#path.append(path_pose1)
for i in range(len(positions_converted1)-1):
    path.append([positions_converted1[i][0], positions_converted1[i][1], positions_converted1[i][2], positions_converted1[i][3], positions_converted1[i][4], positions_converted1[i][5], velocity, acceleration, blend_i])

path.append([positions_converted1[-1][0], positions_converted1[-1][1], positions_converted1[-1][2], positions_converted1[-1][3], positions_converted1[-1][4], positions_converted1[-1][5], velocity, acceleration, 0])
rtde_c.moveL(path)

time.sleep(count/89)

path_2 = []
for i in range(len(positions_converted2)-1):
    path_2.append([positions_converted2[i][0], positions_converted2[i][1], positions_converted2[i][2], positions_converted2[i][3], positions_converted2[i][4], positions_converted2[i][5], velocity, acceleration, blend_i])

path_2.append([positions_converted2[-1][0], positions_converted2[-1][1], positions_converted2[-1][2], positions_converted2[-1][3], positions_converted2[-1][4], positions_converted2[-1][5], velocity, acceleration, blend_3])

rtde_c.moveL(path_2)

#rtde_c.stopScript()

True

In [208]:
# Send a linear path with blending in between - (currently uses separate script)
rtde_c.moveL(path)
rtde_c.stopScript()